In [1]:
import pandas as pd
import numpy as np

off_train = pd.read_csv('E:/tianchi/ccf_offline_stage1_train.csv')
off_test = pd.read_csv('E:/tianchi/ccf_offline_stage1_test_revised.csv')
off_train = off_train.dropna(subset = ['Coupon_id'])

In [2]:
off_train['result'] = off_train[['Coupon_id', 'Date']].apply(lambda x: 1 if pd.notnull(x[0]) and pd.notnull(x[1]) else 0, axis = 1)

In [3]:
off_train.Distance = off_train.Distance.fillna(1)
off_test.Distance = off_test.Distance.fillna(1)

# 用户使用该商家的优惠劵的平均时间,Use_mean_time

In [4]:
import datetime

#判断使用优惠券日期-领取优惠券日期是否小于15天，小于取1，大于取0 
def days(z):
    if str(z['Date_received']) != 'nan' and str(z['Date']) != 'nan':
        days = (datetime.datetime.strptime(str(int(z['Date'])), "%Y%m%d") - datetime.datetime.strptime(str(int(z['Date_received'])), "%Y%m%d"))
        return days.days
    else:
        return np.NaN
    
off_train['days'] =  off_train.apply(days, axis=1)
use_Coupon = off_train.dropna(subset=['days'])

In [5]:
a = use_Coupon['days'].groupby(use_Coupon['User_id']).mean()
a = a.reset_index()
a = a.rename(columns={'days':'Use_mean_time'})

In [6]:
off_train = pd.merge(off_train, a, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, a, on = 'User_id', how = 'left')

# 用户消费的平均距离,Use_mean_Distance

In [7]:
a = use_Coupon['Distance'].groupby(use_Coupon['User_id']).mean()
a = a.reset_index()
a = a.rename(columns={'Distance':'Use_mean_Distance'})

In [8]:
off_train = pd.merge(off_train, a, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, a, on = 'User_id', how = 'left')

# 用户使用某种类型的优惠劵数目/用户领取优惠劵总数,Coupon_use/Coupon_sum

In [9]:
a = use_Coupon[['User_id', 'Discount_rate']].groupby([use_Coupon['User_id'], use_Coupon['Discount_rate']]).count()
a = a.rename(columns={'User_id':'Coupon_use'})
a = a.drop(columns=['Discount_rate'])
a = a.reset_index()

In [10]:
b = off_train['Date_received'].groupby(off_train['User_id']).count()
b = b.reset_index()
b = b.rename(columns={'Date_received':'Coupon_recieved'})

In [11]:
c = pd.merge(a, b, on = 'User_id', how = 'left')

In [12]:
def chufa(z):
    x = z.Coupon_use / z.Coupon_recieved
    return x
c['Coupon_use/Coupon_sum'] = c.apply(chufa, axis=1)

In [13]:
off_train = pd.merge(off_train, c[['User_id', 'Discount_rate', 'Coupon_use/Coupon_sum']], on = ['User_id', 'Discount_rate'], how = 'left')
off_test = pd.merge(off_test, c[['User_id', 'Discount_rate', 'Coupon_use/Coupon_sum']], on = ['User_id', 'Discount_rate'], how = 'left')

# 用户领取某种类型的优惠劵数目/用户领取优惠劵总数,Coupon/Coupon_sum

In [14]:
a = off_train[['User_id', 'Discount_rate']].groupby([use_Coupon['User_id'], use_Coupon['Discount_rate']]).count()
a = a.rename(columns={'User_id':'Coupon_use'})
a = a.drop(columns=['Discount_rate'])
a = a.reset_index()

In [15]:
b = off_train['Date_received'].groupby(off_train['User_id']).count()
b = b.reset_index()
b = b.rename(columns={'Date_received':'Coupon_recieved'})

In [16]:
c = pd.merge(a, b, on = 'User_id', how = 'left')

In [17]:
def chufa(z):
    x = z.Coupon_use / z.Coupon_recieved
    return x
c['Coupon/Coupon_sum'] = c.apply(chufa, axis=1)

In [18]:
off_train = pd.merge(off_train, c[['User_id', 'Discount_rate', 'Coupon/Coupon_sum']], on = ['User_id', 'Discount_rate'], how = 'left')
off_test = pd.merge(off_test, c[['User_id', 'Discount_rate', 'Coupon/Coupon_sum']], on = ['User_id', 'Discount_rate'], how = 'left')

# 用户在某个商铺的优惠券领取数,User_Merchant_Coupon

In [19]:
a = off_train['Date_received'].groupby([off_train['User_id'], off_train['Merchant_id']]).count()
a = a.reset_index()
a = a.rename(columns={'Date_received':'User_Merchant_Coupon'})

In [20]:
off_train = pd.merge(off_train, a, on = ['User_id', 'Merchant_id'], how = 'left')
off_test = pd.merge(off_test, a, on = ['User_id', 'Merchant_id'], how = 'left')

# 用户在某个商铺的优惠券消费数,User_Merchant_Coupon_used

In [21]:
a = off_train['Date'].groupby([off_train['User_id'], off_train['Merchant_id']]).count()
a = a.reset_index()
a = a.rename(columns={'Date':'User_Merchant_Coupon_used'})

In [22]:
off_train = pd.merge(off_train, a, on = ['User_id', 'Merchant_id'], how = 'left')
off_test = pd.merge(off_test, a, on = ['User_id', 'Merchant_id'], how = 'left')

In [23]:
off_train = off_train.fillna(0)
off_test = off_test.fillna(0)

In [29]:
#对训练集和测试集中的Discount_rate进行独热编码
off_train = pd.get_dummies(off_train, prefix=['Discount_rate'])
off_test = pd.get_dummies(off_test, prefix=['Discount_rate'])

#训练集中出现了4个测试集中没有出现的码段，进行填充
m = list(set(off_train.columns.tolist()) - set(off_test.columns.tolist()))
for i in m:
    off_test[i] = 0

#测试集中有一个训练集中没有的码段，进行填充    
off_train['Discount_rate_500:30'] = 0

In [30]:
#训练数据、训练标签、测试数据的设置
x_train = off_train.drop(columns = ['Date_received', 'User_id', 'Merchant_id', 'Coupon_id', 'Date', 'result', 'days'])
y_train = off_train.result
x_test = off_test.drop(columns = ['Date_received', 'User_id', 'Merchant_id', 'Coupon_id'])

#将训练数据和测试数据的列一一对应
x_test = x_test[x_train.columns.tolist()]

from sklearn.linear_model import LogisticRegression

#调用逻辑回归模型，并进行训练
clf = LogisticRegression()
clf.fit(x_train, y_train)

#进行预测，并将需要保存的部分写入save
predict = clf.predict_proba(x_test)
save = off_test[['User_id', 'Coupon_id', 'Date_received']]
save.insert(3, 'probability', predict[:,1], True)

#输出结果为csv
save.to_csv('E:/tianchi/20180725/add_6_feature.csv', index = None)

In [31]:
save

,User_id,Coupon_id,Date_received,probability
0,4129537,9983,20160712,0.009120
1,6949378,3429,20160706,0.994437
2,2166529,6928,20160727,0.002104
3,2166529,1808,20160727,0.003332
4,6172162,6500,20160708,0.009688
5,4005121,9983,20160706,0.012585
6,4347394,9983,20160716,0.016856
7,3094273,13602,20160727,0.012230
8,5139970,9983,20160729,0.023719
9,3237121,13602,20160703,0.009120
